In [8]:
import time
import tkinter as tk 
import numpy as np
import matplotlib.pyplot as plt

class Valve: 
    def __init__(self):
        self.state = "closed"
        self.target_state = "closed"
        self.flowrate = 0.0
        self.transition_time = 5 ##Time to open/close valve in seconds
        self.transition_progress = 0 ##Progress of opening/closing in seconds

    def open(self):
        if self.state == "closed": ##Only open if closed
            self.state = "transitioning"
            self.target_state = "open"
            self.transition_progress = 0

    def close(self):
        if self.state == "open":
            self.state = "transitioning"
            self.target_state = "closed"
            self.transition_progress = 1.0 ##Transition progress is 1.0 when 
            
    def update(self, dt):
        if self.state == "transitioning":
            if self.target_state == "open":
                self.transition_progress += dt / self.transition_time
                if self.transition_progress >= 1.0:
                    self.transition_progress = 1.0
                    self.state = "open"

            elif self.target_state == "closed":
                self.transition_progress -= dt / self.transition_time
                if self.transition_progress <= 0.0:
                    self.transition_progress = 0.0
                    self.state = "closed"
        self.update_flowrate()

    def update_flowrate(self):
        max_flowrate = 0.00072 ##Maximum flowrate in m^3/s
        self.flowrate= self.transition_progress * max_flowrate








    def __str__(self):
        return f"Valve is {self.state} with flowrate {self.flowrate}"



# valve = Valve()
# valve.open()
# dt = 1
# for t in range(30):
#     valve.update(dt)
#     print(f"Time: {t+1}s → {valve}")
#     time.sleep(1)

class ValveGUI:
    def __init__(self, parent, valve):
        self.valve = valve
        self.root = parent  # ✅ Store parent for use in .after()

        frame = tk.LabelFrame(parent, text="Valve", padx=10, pady=10)
        frame.pack(side=tk.LEFT, padx=10, pady=10)

        self.state_label = tk.Label(frame, text="State: ---", font=("Arial", 14))
        self.state_label.pack(pady=5)

        self.flowrate_label = tk.Label(frame, text="Flowrate: 0.00000 m³/s", font=("Arial", 14))
        self.flowrate_label.pack(pady=5)

        self.open_button = tk.Button(frame, text="Open Valve", command=self.open_valve)
        self.open_button.pack(pady=5)

        self.close_button = tk.Button(frame, text="Close Valve", command=self.close_valve)
        self.close_button.pack(pady=5)

        self.canvas = tk.Canvas(frame, width=100, height=100)
        self.canvas.pack(pady=5)
        self.circle = self.canvas.create_oval(20, 20, 80, 80, fill="gray")

        self.update_gui()



    def open_valve(self):
        self.valve.open()

    def close_valve(self):
        self.valve.close()
    
    def update_gui(self):
        self.valve.update(1)  # simulate 1 second passing
        self.state_label.config(text=f"State: {self.valve.state}")
        self.flowrate_label.config(text=f"Flowrate: {self.valve.flowrate:.5f} m³/s")
        self.root.after(1000, self.update_gui) 
        if self.valve.state == "open":
            self.canvas.itemconfig(self.circle, fill="green")
        elif self.valve.state == "closed":
            self.canvas.itemconfig(self.circle, fill="red")
        else:
            self.canvas.itemconfig(self.circle, fill="yellow")




In [9]:
# root = tk.Tk()
# valve = Valve()
# app = ValveGUI(root, valve)
# root.mainloop()




        



In [10]:
class Pump:
    def __init__(self, name="pump", max_flowrate=0.00062, transition_time=5):    
        self.name = name
        self.flowrate = 0.0
        self.max_flowrate = max_flowrate
        self.transition_time = transition_time
        
        self.state = "off"  # Initial state of the pump
        self.transition_progress = 0.0
          # Time to turn on/off the pump in seconds

    def turn_on(self):
        if self.state == "off":
            self.state = "transitioning_on"
            self.transition_progress = 0.0

    def turn_off(self):
        if self.state == "on":
            self.state = "transitioning_off"
            self.transition_progress = 1.0

    def update(self, dt):
        if self.state == "transitioning_on":
            self.transition_progress += dt / self.transition_time
            if self.transition_progress >= 1.0:
                self.transition_progress = 1.0
                self.state = "on"

        elif self.state == "transitioning_off":
            self.transition_progress -= dt / self.transition_time
            if self.transition_progress <= 0.0:
                self.transition_progress = 0.0
                self.state = "off"

    def get_outlet_flowrate(self, dt):
        self.flowrate = self.transition_progress * self.max_flowrate
        return self.flowrate * dt
    

class PumpGUI:
    def __init__(self,parent, pump):
        self.pump = pump

        self.frame = tk.LabelFrame(parent, text="Pump Control", padx=10, pady=10)
        self.frame.pack(side=tk.LEFT, padx=10, pady=10)

        self.status_label = tk.Label(self.frame, text="Status: ---", font=("Arial", 14))
        self.status_label.pack(pady=5)

        self.toggel_button = tk.Button(
            self.frame,
            text = "Toggle Pump",
            command = self.toggle_pump,
            font = ("Arial", 14)
        )
        self.toggel_button.pack(pady=5)

        self.update_gui()

    def toggle_pump(self):
        if self.pump.state in  ["off", "transitioning_off"]: ## if the pump is off or transitioning off, turn it on
            self.pump.turn_on()
        elif self.pump.state in ["on", "transitioning_on"]:
            self.pump.turn_off()

    def update_gui(self):
        self.status_label.config(text=f"State: {self.pump.state} ({self.pump.transition_progress * 100:.0f}%)"
    )
        self.frame.after(1000, self.update_gui)
        if self.pump.state == 'on':
            self.status_label.config(text="Status: ON", fg="green")
        elif self.pump.state == 'off':
            self.status_label.config(text="Status: OFF", fg="red")
        elif self.pump.state == 'transitioning_on':
            self.status_label.config(text="Transitioning ON", fg="orange")
        elif self.pump.state == 'transitioning_off':
            self.status_label.config(text="Transitioning OFF", fg="yellow")
            
            
                
   








In [11]:
class Tank:
    def __init__(self, max_height_m=1200, area_m2=1.0):

        self.level = 500 ##Level in mm
        self.max_level = max_height_m
        self.area = area_m2 
        self.min_level = 0.0 ##Minimum level in m
        # self.chem_conc = { 
        #     "NaCl": 0.0,  # Sodium Chloride
        #     "HCL": 0.0,  # Calcium Chloride
        #     "NaOCl": 0.0,  # Magnesium Sulfate
        # }


        self.ripple_active = False ##Ripple active or not
        self.ripple_time = 0.0 ##Time since ripple started in seconds
        self.ripple_duration = 5.0 ##Duration of ripple in seconds

        self.A0 = 0.001 ##Amplitude of ripple in m
        self.lambda_damp = 40 
        self.k = 2 * np.pi /0.02
        self.omega = np.sqrt(              # Angular frequency of the wave
            self.k * (9.81 + (0.0728 / 1000) * self.k**2) * np.tanh(self.k * self.max_level)  # Wave speed in m/s
        )

        self.pump_on = False ##Pump on or off
        self.outlet_flowrate = 0.00062
        
        self.pumps = []  # List to hold pump objects
        
        
        




    def update(self, inlet_flowrate_m3s, dt):
        added_volume = inlet_flowrate_m3s * dt
        added_height = added_volume / self.area  # This is the normal water rise

    # 2. Compute ripple height if ripple is active
        ripple_height = 0.0
        if self.ripple_active:
            self.ripple_timer += dt  # Advance ripple timer
            t = self.ripple_timer

            if t <= self.ripple_duration:
                    # Compute ripple using your physics-based equation
                ripple_height = self.A0 * np.exp(-self.lambda_damp * t) * np.cos(self.omega * t)
            else:
                self.ripple_active = False  # Stop ripple after 5 seconds

    # 3. Add both ripple and normal rise to water level
        self.level += added_height + ripple_height

        # Check if the pump is on and adjust the water level accordingly
        total_outlet_flowrate = 0.0 # Pump flowrate in m^3/s
        for pump in self.pumps:
            pump.update(dt)
            total_outlet_flowrate += pump.get_outlet_flowrate(dt)

        outlet_height = total_outlet_flowrate / self.area  # Convert flowrate to height
        self.level -= outlet_height  # Decrease level by outlet flowrate


    def get_level_state(self):
        level = self.level ##Convert to mm
        if level < 200:
            return "LL"
        elif level < 500:
            return "L"
        elif level < 800:
            return "H"
        else:
            return "HH"
        
    def check_extremes(self):
        if self.level > self.max_level:
            return "overflow"
        elif self.level < self.min_level:
            return "underflow"
        else:
            return "normal"
        

class TankGUI:
    def __init__(self, parent, tank):
        self.tank = tank

        # Create a labeled frame inside the main window
        frame = tk.LabelFrame(parent, text="Tank 101 Control", padx=10, pady=10)
        frame.pack(side=tk.LEFT, padx=10, pady=10)

        # Label to show the tank level in meters
        self.level_label = tk.Label(frame, text="Level: 0.000 mm", font=("Arial", 14))
        self.level_label.pack(pady=5)

        # Label to show the tank stage (LL, L, H, HH)
        self.stage_label = tk.Label(frame, text="State: ---", font=("Arial", 14))
        self.stage_label.pack(pady=5)

        # Label to show the overflow/underflow status
        self.status_label = tk.Label(frame, text="Status: ---", font=("Arial", 14))
        self.status_label.pack(pady=5)

        self.parent = parent  # Save the parent for `after` scheduling

        self.update_gui()

    def update_gui(self):
        self.level_label.config(text=f"Level: {self.tank.level:.7f} m")
        self.stage_label.config(text=f"Stage: {self.tank.get_level_state()}")
        self.status_label.config(text=f"Status: {self.tank.check_extremes()}")
        if self.tank.check_extremes() == "overflow":
            self.status_label.config(fg="red")
        elif self.tank.check_extremes() == "underflow":
            self.status_label.config(fg="red")
        else:
            self.status_label.config(fg="green")

        # Schedule the next update after 1 second
        self.parent.after(1000, self.update_gui)






In [12]:
# class ChemicalTank:
#     def __init__(self, name, initial_volume):
#         self.name = name
#         self.volume = initial_volume
#         self.remaining_volume = initial_volume

#     def subtract(self, volume_l):
#         if volume_l <= self.remaining_volume:
#             self.remaining_volume -= volume_l
#         else:
#             self.remaining_volume = 0.0  # Prevent negative volume

#     def get_remaining_volume(self):
#         return self.remaining_volume


In [13]:
# class Dosing_pump:
#     def __init__(self, name, flowrate_L_hr, chemical_tank, transition_time=5):
#         self.name = name
#         self.flowrate_L_hr = flowrate_L_hr
#         self.flowrate_L_s = flowrate_L_hr / 3600.0  # Convert to L/s
#         self.chemical_tank = chemical_tank
#         self.transiotion_time = transition_time  # Time to turn on/off the pump in seconds
#         self.state = "off"
#         self.transition_progress = 0.0

#     def turn_on(self):
#         if self.state == "off":
#             self.state = "transitioning_on"
#             self.transition_progress = 0.0
    
#     def turn_off(self):
#         if self.state == "on":
#             self.state = "transitioning_off"
#             self.transition_progress = 1.0
    
#     def update(self, dt):
#         if self.state == "transitioning_on":
#             self.transition_progress += dt / self.transiotion_time
#             if self.transition_progress >= 1.0:
#                 self.transition_progress = 1.0
#                 self.state = "on"

#         elif self.state == "transitioning_off":
#             self.transition_progress -= dt / self.transiotion_time
#             if self.transition_progress <= 0.0:
#                 self.transition_progress = 0.0
#                 self.state = "off"

#     def get_dosed_volume(self, dt):
#         current_flowrate = self.transition_progress * self.flowrate_L_s
#         volume_dosed = current_flowrate*dt # Current flowrate in L/s
#         self.chemical_tank.subtract(volume_dosed)
#         return volume_dosed

In [14]:
root = tk.Tk()
root.title("Water System Simulation")

valve = Valve()
tank = Tank()
pump = Pump()
tank.pumps.append(pump)  # ✅ correct
  # Add the pump to the tank's list of pumps

# valve.open()
# pump.turn_off()

# Pass the same root to both GUIs
valve_gui = ValveGUI(root, valve)
tank_gui = TankGUI(root, tank)
pump_gui = PumpGUI(root, pump)

# Simulation update loop
def loop_update():
    valve.update(1)

    # ✅ Here is where we detect the valve opening and trigger the ripple
    if valve.state == "transitioning" and valve.target_state == "open":
        if not tank.ripple_active:
            tank.ripple_active = True
            tank.ripple_timer = 0.0  # reset the ripple timer

    tank.update(valve.flowrate, 1)

    root.after(1000, loop_update)
    
    
    
loop_update()
root.mainloop()

